In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# !pip install coloredlogs onnx onnxruntime

In [3]:
from flair.models import TextClassifier
from flair.embeddings import TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus, Sentence

In [4]:
model = TextClassifier.load("models/USER_base_1731794531/final-model.pt")

In [5]:
assert isinstance(model.embeddings, (TransformerWordEmbeddings, TransformerDocumentEmbeddings))

In [6]:
column_name_map = {0: "text", 1: "label"}
corpus: Corpus = CSVClassificationCorpus("flair_data",
                                            column_name_map,
                                            skip_header=False,
                                            delimiter='\t',    # tab-separated files
                                            label_type='label')

2024-11-17 00:46:07,967 Reading data from flair_data
2024-11-17 00:46:07,971 Train: flair_data/train.csv
2024-11-17 00:46:07,973 Dev: flair_data/dev.csv
2024-11-17 00:46:07,974 Test: flair_data/test.csv


In [7]:
test = list(corpus.test)[:100]

In [8]:
model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label') # 14 min - corpus.test

  2%|▏         | 1/63 [00:22<22:58, 22.23s/it]

KeyboardInterrupt



In [9]:
model.embeddings = model.embeddings.export_onnx("embeddings-user-base.onnx", test[:10], providers=[ "CPUExecutionProvider"], session_options={})

/opt/conda/lib/python3.10/site-packages/transformers/models/deberta/modeling_deberta.py:646: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  scale = torch.sqrt(torch.tensor(query_layer.size(-1), dtype=torch.float) * scale_factor)
/opt/conda/lib/python3.10/site-packages/transformers/models/deberta/modeling_deberta.py:646: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale = torch.sqrt(torch.tensor(query_layer.size(-1), dtype=torch.float) * scale_factor)
/opt/conda/lib/python3.10/site-packages/transformers/models/deberta/modeling_deberta.py:111: TracerWarning: torch.tensor results are 

In [10]:
model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label') # 3 30 min

  6%|▋         | 4/63 [00:18<04:28,  4.55s/it]

KeyboardInterrupt



In [11]:
model.save("model-onnx-user-base.pt")

In [12]:
model.embeddings.quantize_model(
            "embeddings-user-base-optimized-8bit.onnx",  extra_options={"DisableShapeInference": True}, use_external_data_format=False
        )

In [13]:
model.save("quant_model.pt")

In [ ]:
model = TextClassifier.load("model.pt")

In [ ]:
model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label')

In [ ]:
model.embeddings.onnx_model